In [19]:
import pandas as pd
import numpy as np

test = pd.read_csv("test.csv")
train= pd.read_csv("train.csv")
test_ids = test["PassengerId"]
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Limpeza dos Data Frame

In [20]:
df1=train.drop(['Name',"PassengerId",'Ticket','Cabin'], axis=1)
testData = test.drop(['Name',"PassengerId", 'Ticket', 'Cabin'], axis=1)
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [35]:
def clean(tenta):
    x = tenta.drop(["Ticket","Cabin", "Name", "PassengerId"], axis=1)
    colunas = ["SibSp", "Parch" , "Fare" , "Age"]

    for coluna in colunas:
        x[coluna].fillna(x[coluna].median(), inplace=True)

    x.Embarked.fillna("U", inplace=True)
    return x

train= clean(train)
test=clean(test)
train.head()

KeyError: "['Ticket', 'Cabin', 'Name', 'PassengerId'] not found in axis"

In [22]:
# Função para verificar valores nulos no DataFrame
def verificaNulos(dataframe):
    valoresNulos = dataframe.isnull().sum()
    print('\n\nValores Nulos:\n', valoresNulos)

In [23]:
verificaNulos(df1)



Valores Nulos:
 Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [24]:
verificaNulos(testData)



Valores Nulos:
 Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64


### Limpeza dos dados nulos

In [25]:
# Variaveis para armazenar a mediana da idade de cada sexo
idadeMediaHomem = df1[df1['Sex']==1]['Age'].median()
idadeMediaMulher = df1[df1['Sex']==0]['Age'].median()
# Atribui o valor da mediana aos passageiros que possuem a idade como nula
df1.loc[(df1.Age.isnull()) & (df1['Sex']==1),'Age'] = idadeMediaHomem
df1.loc[(df1.Age.isnull()) & (df1['Sex']==0),'Age'] = idadeMediaMulher
#------------------------------------------------------------------------------------------
idadeMediaHomem1 = testData[testData['Sex'] == 1]['Age'].median()
idadeMediaMulher1 = testData[testData['Sex'] == 0]['Age'].median()
testData.loc[(testData.Age.isnull()) & (testData['Sex'] == 1), 'Age'] = idadeMediaHomem1
testData.loc[(testData.Age.isnull()) & (testData['Sex'] == 0), 'Age'] = idadeMediaMulher1

verificaNulos(df1)

In [26]:
# verificando quais são os dois passassageiros com embarked nulo
EmbarkedNulo = df1[df1.Embarked.isnull()]
EmbarkedNulo

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
61,1,1,female,38.0,0,0,80.0,NaN
829,1,1,female,62.0,0,0,80.0,NaN


In [27]:
# Atribuindo um valor
df1 = df1.fillna(2)

In [28]:
# Preenchendo valores nulos na tarifa com a mediana
tarifaMedia = testData['Fare'].median()
testData.loc[testData.Fare.isnull(), 'Fare'] = tarifaMedia

In [29]:
verificaNulos(df1)



Valores Nulos:
 Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


In [33]:
verificaNulos(testData)



Valores Nulos:
 Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         0
Embarked     0
dtype: int64


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Y = train["Survived"]
X = train.drop("Survived", axis=1)

X_train, X_val, y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

clf= LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
a= accuracy_score(Y_val, predictions)
print(a*100)

ValueError: could not convert string to float: 'male'

In [16]:
submission_preds = clf.predict(test)

df= pd.DataFrame({"PassengerId": test_ids, 
                  "Survived": submission_preds})
df.to_csv("submission.csv", index=False)